# Weapon target assignment

Versione WTA con **ricerca esaustiva**

---
Per commenti, modifiche et al. contattateci via email:
- marco.vannucci@santannapisa.it
- valentina.colla@santannapisa.it

## Importazioni e variabili globali

In [1]:
import numpy as np
hc=0 ## questa è una variabile globale

## Create particle class

In [2]:
class Particle:
    def __init__(self):
        self.position = []
        self.velocity = []
        self.value = []
        self.best_position = []
        self.best_value = []
    

## Funzioni accessorie

### Calcolo possibili combinazioni *weapon-target*

La funzione `combina` calcola tutte le possibili combinazioni di `N_weapon` armi su `N_target` bersagli e le mette nella matrice `all_mat`, le cui dimensioni sono note e precedentemente calcolate, essendo le combinazioni `N_target^N_weapon`

In [3]:
def combina(lista_b, N_weapon, N_target, out_mat, out_pos, all_mat):
    global hc
    for k in range(N_target):
        out_mat[out_pos]=lista_b[k]
        if N_weapon>1:
            combina(lista_b,N_weapon-1,N_target,out_mat,out_pos+1, all_mat)
        else:
            #print(out_mat) serve a visualizzare tutte le possibili combinazioni armi-bersaglio
            all_mat[hc]=out_mat
            hc=hc+1
    return

### Funzione obiettivo

La funzione `valuta_comb` calcola la funzione obiettivo da **minimizzare** per la combinazione armi-bersagli, contenuta nel vettore `combin`, che è il primo argomento.

La funzione obiettivo è la somma dei prodotti del valore di ciascun bersaglio moltiplicato per la probabilità  che esso sopravviva alle armi che sono a lui assegnate, che è *1 - la probabilità che l'arma colpisca il bersaglio*. 

La matrice delle probabilità che ciascun tipo di arma ha di colpire il bersaglio è uno degli argomenti, ossia `Mat_prob`. 
Fra gli argomenti passo anche il vettore `vett_W` che contiene il tipo di ciascuna delle armi della combinazione.
la funzione di compone di due cicli annidati.

**I cicli:**
- Il ciclo esterno itera sui bersagli e quindi calcola i singoli addendi della funzione obiettivo.
- Il ciclo interno itera sulle armi e serve a calcolare la probabailità che ciascun bersaglio sopravvivva all'attacco, che è una produttoria delle probabilità che ha di sopravvivere a ciascuna delle armi. Se un'arma è assegnata proprio a lui, il fattore della produttoria è *(1-p)*, se invece non lo è, il fattore della produttoria è *1* e  quindi, in pratica, non si fa alcuna moltiplicazione.


In [4]:
def valuta_comb(combin, vett_W, Mat_prob, valori_T):
    valore=0
    n_T=len(valori_T)
    n_W=len(combin)
    for i in range(n_T): # il ciclo esterno calcola ogni addendo della funzione obiettivo da minimizzare, relativo a ciascun target
        prob_sopravv_target=1;
        for j in range(n_W):
            if combin[j]==int(i+1): # Il ciclo interno calcola la probabilità  di sopravvivenza di ciascun target 
                prob_sopravv_target*=(1-Mat_prob[vett_W[j]-1,i])
        valore+=valori_T[i]*prob_sopravv_target
    return valore

## Create population

In [5]:
def create_population(nPop, n_bersagli, vett_W, Mat_prob, valori_T):
    W_tot = len(vett_W)
    population = [Particle() for i in range(nPop)]
    global_best = [[], [np.Inf]]
    for i in range(nPop):
        population[i].position = np.random.uniform(1, n_bersagli+1, (W_tot)).astype(int)
        population[i].velocity = np.zeros(W_tot)
        population[i].value = valuta_comb(population[i].position, vett_W, Mat_prob, valori_T)
        
        # Update personal
        population[i].best_position = population[i].position
        population[i].best_value = population[i].value
        
        # Update global
        if population[i].best_value < global_best[1]:
            global_best = [population[i].best_position, population[i].best_value]
    
    return population, global_best

## Update population

In [6]:
def update_pop(population, w, c1, c2, min_velocity, max_velocity, n_bersagli, global_best, vett_W, Mat_prob, valori_T):
    for i in range(len(population)):
        
        # Update velocity
        population[i].velocity = w*population[i].velocity + c1*np.random.uniform()*(population[i].best_position - population[i].position) + c2*np.random.uniform()*(global_best[0] - population[i].position)
                
        # Velocity limits
        population[i].velocity = np.clip(population[i].velocity, min_velocity, max_velocity)
        
        # Update position
        population[i].position = population[i].position + population[i].velocity
        
        # Bounds
        population[i].position = np.clip(population[i].position, 1, n_bersagli)
        
        population[i].position = np.rint(population[i].position)
        # Evaluate
        population[i].value = valuta_comb(population[i].position, vett_W, Mat_prob, valori_T)
        
        # Update personal best
        if population[i].value < population[i].best_value:
            population[i].best_position = population[i].position
            population[i].best_value = population[i].value
            
        # Update global best
        if population[i].value < global_best[1]:
            global_best = [population[i].best_position, population[i].value]
            
        
    return population, global_best

## Main

Parametri specificati dall'utente:
- il numero dei bersagli
- il numero di tipi di armi.

Parametri settati automaticamente:
- il valore dei bersagli è un valore intero casuale tra 0 e 100
- la probabilità che un tipo di arma colpisca ciascun bersaglio è anche essa casuale

**Codifica soluzione**

Ogni combinazione armi-bersagli è codificata come una *stringa di interi* di lunghezza pari al numero totale di armi disponibili, **nell'ordine in cui i tipi vengono forniti**.

> **Ad esempio**: se io ho 2 armi di tipo A e 3 armi di tipo B, la combinazione è lunga 5 e così composta [A1, A2, B1, B2, B3]

Il contenuto di ciascun elemento è il numero del target a cui l'arma corrispondente è assegnata.
Chiaramente, armi dello stesso tipo hanno la medesima probabilità  di colpire ogni bersaglio.
La matrice delle probabilità  che ciascun tipo di arma ha di colpire ogni bersaglio si chiama `Mat_prob_cogliere_bersagli` ed ha tante righe quante sono i tipi di armi e tante colonne quante sono i bersagli.

> Ho creato anche una lista che ha tante righe quane i tipi di armi, che è una lista mista, che contiene per ogni riga il nome del tipo di arma, il numero di armi di quel tipo e il vettore delle probabilità  che questo tipo di arma colpisca i diversi bersagli.
Questa lista non serve assolutamente a niente, l'ho creata per sfizio e come struttura dati che raccoglie tutte le informazioni relative alle ami. 

In [7]:
n_bersagli=int(input('Quanti bersagli ci sono? ') )
#lista_bersagli = [1, 2, 3]
lista_bersagli=list(range(1,n_bersagli+1))

# determino casualmente il valore di ciascun bersaglio
Valori_bersagli = np.array(100*np.random.rand(n_bersagli),dtype=int)

print('\n%12s   %8s'%('Bersaglio','Valore'))
for i in lista_bersagli:
    print('%12d   %8d'%(i,Valori_bersagli[i-1]))


   Bersaglio     Valore
           1         53
           2          4
           3         82
           4         27
           5         46


In [8]:
# inserimento tipologia e delle armi
n_tipo_armi=int(input('Quanti tipi di armi ci sono? ') )

# numero di armi per ciascun tipo 
cont_armi_per_tipo=np.zeros(n_tipo_armi, dtype=int)

# matrice con la probabilità, per ciascuna tipologia di arma, di colpire ciascun bersaglio
Mat_prob_cogliere_bersagli=np.zeros([n_tipo_armi,n_bersagli])

# descrittore del set di armi (include nome, tipo e le probabilità di colpire i bersagli)
lista_armi=[] 


for j in range(n_tipo_armi):
    nome = 'WEATYPE_%d'%j
    probab_cogliere_bersagli=np.random.rand(n_bersagli)
    Mat_prob_cogliere_bersagli[j]=probab_cogliere_bersagli
    testo ='Quante armi ci sono del tipo '+ nome + '? '
    cont_armi_per_tipo[j]=input(testo)
    lista_armi.append([nome, cont_armi_per_tipo[j],probab_cogliere_bersagli])

    
#for l in lista_armi:
#    print(l)

### Simulazione delle combinazioni

In [9]:
W_tot=int(np.sum(cont_armi_per_tipo)) #numero totale delle armi
Vec_tipo_armi=np.zeros(W_tot,dtype=int) #vettore con il tipo di ciascuna arma
offset=0
for i in range(n_tipo_armi):
    for j in range(cont_armi_per_tipo[i]):
        Vec_tipo_armi[j+offset]=i+1;
    offset+=cont_armi_per_tipo[i]
    
# adesso faccio decidere all'utente se andare avanti con il calcolo in base al numero effettivo delle combinazioni da valutare
N_comb=int(n_bersagli**W_tot) #numero totale delle combinazioni armi-bersagli

print('Saranno valutate %d combinazioni arma-target'%N_comb)
cont=input('Continuare? [s/n]')

print(cont)

Saranno valutate 9765625 combinazioni arma-target
s


In [10]:
if cont=='s':
    Tutte_combin=np.zeros([N_comb, W_tot], dtype=int) #array che contiene tutte le associazioni possibili delle armi ai bersagli
    assoc=np.zeros(W_tot)
    out_pos=0
    combina(lista_bersagli, W_tot, n_bersagli, assoc, out_pos, Tutte_combin)

    Valore_combin=np.zeros(N_comb)
    for i in range(N_comb):
        Valore_combin[i]=valuta_comb(Tutte_combin[i],Vec_tipo_armi,Mat_prob_cogliere_bersagli,Valori_bersagli)
    indx_Best=np.argmin(Valore_combin)
    Best_combin=Tutte_combin[indx_Best]
    Min_obj=min(Valore_combin)
    print("la migliore combinazione è ", Best_combin)
    print("Il valore della funzione obiettivo in ", Best_combin, " è ", Min_obj)

la migliore combinazione è  [2 5 5 3 4 1 1 3 1 4]
Il valore della funzione obiettivo in  [2 5 5 3 4 1 1 3 1 4]  è  8.346617564688685


## PSO solution

In [11]:
## Constriction coefficients
kappa = 1
phi1 = 2.05
phi2 = 2.05
phi = phi1 + phi2
chi = 2*kappa/abs(2-phi-np.sqrt(phi*phi-4*phi))

In [12]:
## Hyperparameters
w = chi
c1 = chi*phi1
c2 = chi*phi2
max_velocity = 0.2*(n_bersagli-1)
min_velocity = -max_velocity
n_iter = 100
wdamp = 1
nPop = 50

In [13]:
population, global_best = create_population(nPop, n_bersagli, Vec_tipo_armi, Mat_prob_cogliere_bersagli, Valori_bersagli)
print(global_best)

[array([5, 4, 3, 3, 3, 5, 1, 3, 4, 1]), 14.607243763351969]


In [14]:
## Vector of best per iter
best_costs = np.zeros((n_iter, 1))

In [15]:
for i in range(n_iter):
    population, global_best= update_pop(population, w, c1, c2, min_velocity, max_velocity, n_bersagli, global_best, Vec_tipo_armi, Mat_prob_cogliere_bersagli, Valori_bersagli)
    best_costs[i, 0] = global_best[1]
    w = wdamp*w
    

In [16]:
print(global_best)

[array([5., 5., 2., 4., 3., 5., 1., 3., 4., 1.]), 10.5254051984084]


In [17]:
print(best_costs)

[[11.32921319]
 [11.32921319]
 [11.09606606]
 [11.09606606]
 [11.09606606]
 [11.09606606]
 [11.09606606]
 [11.09606606]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.5254052 ]
 [10.52540

## Hyperparameter tuning using Bayesian Optimization

In [68]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
SMOKE_TEST = os.environ.get("SMOKE_TEST")

In [69]:
from botorch.models import FixedNoiseGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood

bounds = np.array(([0, 0, 0, 0, 0.5, 50], [1, 10, 10, 5, 1, 1000]), dtype=float)
train_X = np.array([w, c1, c2, max_velocity, wdamp, nPop])
for i in range(9):
    train_X = np.vstack((train_X, np.random.uniform(0,1, (1, 6))*(bounds[1,:] - bounds[0, :])+bounds[0,:]))


In [70]:
train_Y = np.zeros((10,1))
for i in range(len(train_X)):
    pop, glob = create_population(train_X[i, 5].astype(int), n_bersagli, Vec_tipo_armi, Mat_prob_cogliere_bersagli, Valori_bersagli)
    for j in range(n_iter):
        pop, glob= update_pop(pop, train_X[i, 0], train_X[i, 1], train_X[i, 2], -train_X[i, 3], train_X[i, 3], n_bersagli, glob, Vec_tipo_armi, Mat_prob_cogliere_bersagli, Valori_bersagli)
        w = w * train_X[i, 4]
    train_Y[i] = glob[1]

In [72]:
print(train_Y)

[[13.13405093]
 [16.18518796]
 [ 8.83464015]
 [ 8.51619464]
 [ 8.34661756]
 [ 8.34661756]
 [ 9.66558846]
 [ 8.51619464]
 [ 8.51619464]
 [12.80386259]]


In [73]:
# Transfer to tensors
train_X = torch.from_numpy(train_X)
train_Y = torch.from_numpy(train_Y)
bounds = torch.from_numpy(bounds)

In [74]:
# Fit the noisy model
NOISE_SE = 0.2
train_Yvar = torch.tensor(NOISE_SE**2, device=device, dtype=dtype)
gp = FixedNoiseGP(train_X, train_Y, train_Yvar.expand_as(train_Y))
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)

ExactMarginalLogLikelihood(
  (likelihood): FixedNoiseGaussianLikelihood(
    (noise_covar): FixedGaussianNoise()
  )
  (model): FixedNoiseGP(
    (likelihood): FixedNoiseGaussianLikelihood(
      (noise_covar): FixedGaussianNoise()
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): MaternKernel(
        (lengthscale_prior): GammaPrior()
        (raw_lengthscale_constraint): Positive()
        (distance_module): Distance()
      )
      (outputscale_prior): GammaPrior()
      (raw_outputscale_constraint): Positive()
    )
  )
)

In [77]:
# Acquisition function and optimization
from botorch.acquisition import NoisyExpectedImprovement
from botorch.optim import optimize_acqf


for i in range(30):
    nEI = NoisyExpectedImprovement(gp, train_X)
    candidate_t, acq_value = optimize_acqf(
    nEI, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)
    candidate_t = candidate_t.detach()
    train_X = torch.vstack((train_X, candidate_t))
    candidate = candidate_t.numpy()
    pop, glob = create_population(candidate[0][5].astype(int), n_bersagli, Vec_tipo_armi, Mat_prob_cogliere_bersagli, Valori_bersagli)
    for j in range(n_iter):
        pop, glob= update_pop(pop, candidate[0][0], candidate[0][1], candidate[0][2], -candidate[0][3], candidate[0][3], n_bersagli, glob, Vec_tipo_armi, Mat_prob_cogliere_bersagli, Valori_bersagli)
        w = w * candidate[0][4]
    train_Y = torch.vstack((train_Y, torch.tensor(glob[1])))

    # re-fit the model
    gp = FixedNoiseGP(train_X, train_Y, train_Yvar.expand_as(train_Y))
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll)

In [78]:
# Training seems to go on in BO manner
print(train_Y)

tensor([[13.1341],
        [16.1852],
        [ 8.8346],
        [ 8.5162],
        [ 8.3466],
        [ 8.3466],
        [ 9.6656],
        [ 8.5162],
        [ 8.5162],
        [12.8039],
        [ 8.5162],
        [ 8.5162],
        [ 8.5162],
        [11.2639],
        [ 8.5162],
        [ 8.5162],
        [ 9.4377],
        [13.4271],
        [ 8.3466],
        [22.1873],
        [ 8.6898],
        [ 9.0641],
        [ 9.3036],
        [ 8.8970],
        [ 8.5162],
        [ 8.3466],
        [ 8.5162],
        [11.1580],
        [12.2519],
        [ 8.6898],
        [ 8.3466],
        [ 8.3466],
        [ 8.3466],
        [ 8.3466],
        [12.5407],
        [11.7083],
        [ 8.5162],
        [12.7164],
        [ 8.5162],
        [ 8.5162],
        [ 8.3466]], dtype=torch.float64)


In [79]:
# Let's find an optimal candidate with least nPop
train_Y_np = train_Y.detach().numpy()
optimals_idx = np.array([i for i in range(len(train_Y)) if np.isclose(train_Y_np[i], train_Y_np.min())])
train_X_np = train_X.detach().numpy()
values = np.array([train_X_np[i, 5] for i in optimals_idx])
matrix = np.vstack((optimals_idx, values))
matrix = matrix[:, matrix[1].argsort()]
print(matrix)

[[ 32.          33.           4.           5.          25.
   18.          30.          40.          31.        ]
 [445.43473315 459.4606603  676.7616871  703.10703965 753.69915068
  879.28522401 950.65542189 956.18541422 960.81941724]]


In [84]:
final_params = train_X_np[32]
print(final_params)

[  0.89117118   6.5218407    9.13396932   1.97656681   0.74786783
 445.43473315]


## Final loop with tuned params

In [83]:
# It can still fail sometimes, but increasing the population siza should fix the problem
population_final, global_best_final = create_population(int(final_params[5]), n_bersagli, Vec_tipo_armi, Mat_prob_cogliere_bersagli, Valori_bersagli)
best_costs_final = np.zeros((n_iter, 1))
print(global_best_final[1])
for j in range(n_iter):
        population_final, global_best_final= update_pop(population_final, final_params[0], final_params[1], final_params[2], -final_params[3], final_params[3], n_bersagli, global_best_final, Vec_tipo_armi, Mat_prob_cogliere_bersagli, Valori_bersagli)
        best_costs_final[j, 0] = global_best_final[1]
        print(global_best_final[1])
        w = w * final_params[4]

19.89022799294612
17.125129218486798
13.565006107037082
13.565006107037082
11.40483812962702
9.064053950995966
9.064053950995966
9.064053950995966
9.064053950995966
9.064053950995966
9.064053950995966
9.064053950995966
9.064053950995966
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.834640147389699
8.346617564688685
8.346617564688685
8.346617564688685
8.346617564688685
8.346617564688685
8.346617564688685
8.346617564688685
8.34661